### Import

In [1]:
%load_ext autoreload
%autoreload 2
import world_cup_helpers
import world_cup_2018_info

import re
import pandas as pd
from datetime import datetime

Using TensorFlow backend.


### Download schedule

In [2]:
world_cup_2018 = pd.read_html('https://www.independent.co.uk/sport/football/world-cup/world-cup-2018-full-schedule-groups-fixtures-kick-off-times-venues-matches-wallchart-a8087881.html')[0]

In [3]:
world_cup_2018.head()

,0,1,2,3
0,GROUP STAGE,NaN,NaN,NaN
1,Date,Match,Venue,Kick-off (BST)
2,Thursday June 14,Russia v Saudi Arabia (A),Moscow (Luzhniki),4pm
3,Friday June 15,Egypt v Uruguay (A),Ekaterinburg,1pm
4,Friday June 15,Morocco v Iran (B),St Petersburg,4pm


### Format dataframe

In [4]:
world_cup_2018.columns = world_cup_2018.iloc[1]
world_cup_2018 = world_cup_2018.drop(1)
world_cup_2018 = world_cup_2018.dropna(how='any')
world_cup_2018 = world_cup_2018.reset_index(drop=True)
world_cup_2018.index = range(1, len(world_cup_2018) + 1)

In [5]:
world_cup_2018['date'] = world_cup_2018.Date.apply(lambda x: '2018-' + datetime.strptime(x, '%A %B %d').strftime('%m-%d'))
world_cup_2018['home_team'] = world_cup_2018.Match.str.extract(r'(.*(?= v ))')
world_cup_2018['home_team'] = world_cup_2018.home_team.apply(lambda team: world_cup_helpers.country_mapping.get(team, team))
world_cup_2018['away_team'] = world_cup_2018.Match.str.extract(r'((?<= v ).*?(?= \(|$))')
world_cup_2018['away_team'] = world_cup_2018.away_team.apply(lambda team: world_cup_helpers.country_mapping.get(team, team))
world_cup_2018['home_score'] = None
world_cup_2018['away_score'] = None
world_cup_2018['tournament'] = 'Fifa World Cup'
world_cup_2018['city'] = world_cup_2018.Venue
world_cup_2018['country'] = 'Russia'
world_cup_2018['year'] = 2018

world_cup_2018['pred_home_score'] = None
world_cup_2018['pred_away_score'] = None
for i in range(1, 7):
    world_cup_2018['round_{}'.format(i)] = None
world_cup_2018['experience'] = None
world_cup_2018['age'] = None
world_cup_2018['game_day'] = None

world_cup_2018['group'] = world_cup_2018.Match.str.extract(r'((?<=\().*(?=\)))')

world_cup_2018 = world_cup_2018.drop(columns=['Date', 'Match', 'Venue', 'Kick-off (BST)'])

In [6]:
world_cup_2018.head()

1,date,home_team,away_team,home_score,away_score,tournament,city,country,year,pred_home_score,...,round_1,round_2,round_3,round_4,round_5,round_6,experience,age,game_day,group
1,2018-06-14,Russia,Saudi Arabia,None,None,Fifa World Cup,Moscow (Luzhniki),Russia,2018,None,...,None,None,None,None,None,None,None,None,None,A
2,2018-06-15,Egypt,Uruguay,None,None,Fifa World Cup,Ekaterinburg,Russia,2018,None,...,None,None,None,None,None,None,None,None,None,A
3,2018-06-15,Morocco,Iran,None,None,Fifa World Cup,St Petersburg,Russia,2018,None,...,None,None,None,None,None,None,None,None,None,B
4,2018-06-15,Portugal,Spain,None,None,Fifa World Cup,Sochi,Russia,2018,None,...,None,None,None,None,None,None,None,None,None,B
5,2018-06-16,France,Australia,None,None,Fifa World Cup,Kazan,Russia,2018,None,...,None,None,None,None,None,None,None,None,None,C


### Group stage

In [7]:
group_stage = world_cup_2018[:48].copy()
group_stage['experience'] = group_stage.apply(world_cup_helpers.get_experience, axis=1)
group_stage['age'] = group_stage.apply(world_cup_helpers.get_age, axis=1)
group_stage['host_team'] = group_stage.apply(world_cup_helpers.get_host_team, axis=1)
group_stage['game_day'] = world_cup_helpers.calculate_round(group_stage) + 1

##### Pickle

In [8]:
group_stage.to_pickle('world_cup_2018_group_stage.pkl')

### Round of 16

Sought output format:

In [9]:
round_of_16 = world_cup_2018[48:56]
round_of_16 = round_of_16.apply(lambda row: (row.home_team[1], int(row.home_team[0]), row.away_team[1], int(row.away_team[0])), axis=1)
round_of_16.index = range(1, len(round_of_16) + 1)
round_of_16.to_dict()

{1: ('C', 1, 'D', 2),
 2: ('A', 1, 'B', 2),
 3: ('B', 1, 'A', 2),
 4: ('D', 1, 'C', 2),
 5: ('E', 1, 'F', 2),
 6: ('G', 1, 'H', 2),
 7: ('F', 1, 'E', 2),
 8: ('H', 1, 'G', 2)}

Manually add this to world_cup_2018_info.py

Sought output format:

In [10]:
round_of_16 = world_cup_2018[48:56]
print(round_of_16.date.tolist())

['2018-06-30', '2018-06-30', '2018-07-01', '2018-07-01', '2018-07-02', '2018-07-02', '2018-07-03', '2018-07-03']


Manually add this to world_cup_2018_info.py

### Quarter-finals

Sought output format:

In [11]:
quarter_finals = world_cup_2018.loc[57:60]
quarter_finals = quarter_finals.apply(lambda row: (int(re.findall('\d+', row.home_team)[0]) - 48,
                                  int(re.findall('\d+', row.away_team)[0]) - 48), axis=1)
quarter_finals.index = list('ABCD')
quarter_finals.to_dict()

{'A': (1, 2), 'B': (5, 6), 'C': (7, 8), 'D': (3, 4)}

Manually add this to world_cup_2018_info.py

Sought output format:

In [12]:
quarter_finals = world_cup_2018.loc[57:60]
print(quarter_finals.date.tolist())

['2018-07-06', '2018-07-06', '2018-07-07', '2018-07-07']


Manually add this to world_cup_2018_info.py

### Semi-finals

Sought output format:

In [13]:
semi_finals = world_cup_2018.loc[61:62]
semi_finals = semi_finals.apply(lambda row: (chr(int(re.findall('\d+', row.home_team)[0]) + 8),
                                             chr(int(re.findall('\d+', row.away_team)[0]) + 8)), axis=1)
semi_finals.index = [1, 2]
semi_finals.to_dict()

{1: ('A', 'B'), 2: ('C', 'D')}

Manually add this to world_cup_2018_info.py

Sought output format:

In [14]:
semi_finals = world_cup_2018.loc[61:62]
print(semi_finals.date.tolist())

['2018-07-10', '2018-07-11']


Manually add this to world_cup_2018_info.py

### Finals

Sought output format:

In [15]:
finals = world_cup_2018[-2:]
finals

1,date,home_team,away_team,home_score,away_score,tournament,city,country,year,pred_home_score,...,round_1,round_2,round_3,round_4,round_5,round_6,experience,age,game_day,group
63,2018-07-14,NaN,NaN,None,None,Fifa World Cup,St Petersburg,Russia,2018,None,...,None,None,None,None,None,None,None,None,None,NaN
64,2018-07-15,NaN,NaN,None,None,Fifa World Cup,Moscow (Luzhniki),Russia,2018,None,...,None,None,None,None,None,None,None,None,None,NaN


Since there is not enough info in the dataframe, just create the dict manually:

In [16]:
finals = {
    'bronze': (1, 2),
    'gold': (1, 2)
}
finals

{'bronze': (1, 2), 'gold': (1, 2)}

Manually add this to world_cup_2018_info.py

Sought output format:

In [17]:
finals = world_cup_2018[-2:]
print(finals.date.tolist())

['2018-07-14', '2018-07-15']


Manually add this to world_cup_2018_info.py